In [1]:
from keras.datasets import cifar10
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
from tensorflow_weave import *

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
print(x_train.shape)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
print(y_train.shape)

(50000, 3, 32, 32)
(50000, 10)


In [45]:
filter_size = 3
num_filters = 10
include_center = 0 
pad_size = filter_size//2
mid_layer = 30

In [48]:
inputs = Input(shape=(3,32,32))
tf.cast(inputs, dtype=tf.float64)
######### Layer 1
x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)


x_per = Conv2D(num_filters,
               kernel_size = (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation='relu')(x_pad)

x_loc = Conv2D(num_filters,
               kernel_size= (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x_pad)

x_zero = ZeroWeave()(x_loc)
x_weave = ArrayWeave(include_center = include_center)(x_per)

x = Add()([x_weave, x_zero])

x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

x = Conv2D(int(0.5*num_filters),
           kernel_size= (filter_size,filter_size),
           strides=(filter_size,filter_size),
           padding='valid',
           activation = 'relu')(x)
x = MaxPool2D()(x)

x = Flatten()(x)
x = Dense(mid_layer, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [49]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
model.fit(x_train,y_train, epochs = 10, verbose = 1)

Epoch 1/10
25984/50000 [==============>...............] - ETA: 5:26 - loss: 14.4984 - acc: 0.1004

KeyboardInterrupt: 

In [41]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 38s 4ms/step


[1.4139167850494385, 0.5147]

In [23]:
model.layers

In [25]:

model.layers[2].get_weights()

[array([[[[ 0.29960608, -0.28861186,  0.1592465 ,  0.24725753],
          [-0.24199893,  0.01990325, -0.24087994, -0.2327908 ],
          [ 0.13462713,  0.06380621, -0.05194268, -0.18662104]],
 
         [[-0.05814334, -0.1169623 , -0.08285387, -0.2757885 ],
          [-0.05679119,  0.11527789,  0.13418324,  0.0952201 ],
          [-0.0026477 ,  0.13327926,  0.27648175, -0.2354526 ]],
 
         [[-0.21779808, -0.04931591,  0.10018279, -0.00555757],
          [ 0.08576497,  0.0016393 ,  0.2717865 ,  0.26411957],
          [-0.0416683 , -0.29627255,  0.29565087,  0.04526897]]],
 
 
        [[[-0.04062799,  0.14626591, -0.06930771, -0.270012  ],
          [-0.04287893, -0.20437771, -0.06653652, -0.1122063 ],
          [-0.17345144,  0.02114441,  0.11052226, -0.06783617]],
 
         [[-0.06887268,  0.12182418,  0.0584613 , -0.19139285],
          [ 0.23886715, -0.21013212, -0.18156159,  0.03146605],
          [ 0.10928108, -0.26527032, -0.30255297, -0.09012193]],
 
         [[-0.13999303

In [29]:
model.layers[8].get_weights()

[array([[[[-0.17659913,  0.04425956,  0.03592607, -0.00823027,
           -0.06430855, -0.11623493,  0.19238359, -0.00555114,
            0.08804604,  0.11232868,  0.1582436 , -0.19912536],
          [-0.03459128,  0.18392497,  0.08043488,  0.03508746,
            0.15871221, -0.13208997, -0.14473063, -0.09323473,
            0.05682389,  0.12338021,  0.00291273,  0.16889282],
          [-0.09569734,  0.04555588,  0.16159618, -0.07774577,
            0.03380111, -0.13233125, -0.09905578,  0.18259785,
           -0.12220959,  0.11689623,  0.1155142 , -0.01528297],
          [ 0.05878767,  0.06590427, -0.16495606,  0.18004964,
           -0.08915877, -0.1324701 , -0.1172122 ,  0.02378388,
            0.12347722, -0.05788571, -0.12921175, -0.03900485]],
 
         [[-0.18797381, -0.16916938,  0.134841  ,  0.14691517,
            0.10628194, -0.2006874 ,  0.09512314, -0.14029087,
            0.0990786 ,  0.06301841, -0.14158961, -0.04448771],
          [ 0.0360195 , -0.06677677,  0.0038851